09/17:

In [2]:
import requests, json
import bs4
import re

In [3]:
url = "http://www.librarius.com/gy.htm"
request = requests.get(url)
raw = request.content

In [3]:
#print(raw)

In [4]:
soup = bs4.BeautifulSoup(raw, 'html.parser')

In [5]:
entries = soup('p', limit=1)
text = str(entries)
mylist = text.split('\n')
print(len(mylist))
#print(text)

2050


In [6]:
print(mylist[:10])

['[<p>', '<a name="abegge"><b>abegge</b> <small><i>verb</i></small> pay for (it)<p>', '<a name="abyde"><b>abid, abyd, abyde</b> <small><i>verb, prsnt.</i></small> remain, await, wait; <b>abood</b> <small><i>verb, pst.</i></small> awaited, remained<p>', '<a name="abideth"><b>abideth, abydeth</b> <small><i>verb</i></small> awaits<p>', '<a name="abidyng"><b>abidyng</b> <small><i>verb</i></small> awaiting<p>', '<a name="able"><b>able</b> <small><i>adj.</i></small> suitable<p>', '<a name="abluciouns"><b>abluciouns</b> <small><i>noun</i></small> cleansings<p>', '<a name="abood"><b>abood</b> <small><i>noun</i></small> delay<p>', '<a name="above"><b>above</b> <small><i>adj.</i></small> superior<p>', '<a name="abregge"><b>abregge</b> <small><i>verb</i></small> abridge, shorten<p>']


In [7]:
first_entry = soup.find('a', attrs={'name': re.compile('.*')})
first_entry = str(first_entry)

In [8]:
actual_first = first_entry.split('\n')[0]

In [9]:
print(actual_first)

<a name="abayst"><b>abayst</b> embarrassed<p>


In [11]:
mylist[0] = actual_first

In [12]:
print(mylist[:10])

['<a name="abayst"><b>abayst</b> embarrassed<p>', '<a name="abegge"><b>abegge</b> <small><i>verb</i></small> pay for (it)<p>', '<a name="abyde"><b>abid, abyd, abyde</b> <small><i>verb, prsnt.</i></small> remain, await, wait; <b>abood</b> <small><i>verb, pst.</i></small> awaited, remained<p>', '<a name="abideth"><b>abideth, abydeth</b> <small><i>verb</i></small> awaits<p>', '<a name="abidyng"><b>abidyng</b> <small><i>verb</i></small> awaiting<p>', '<a name="able"><b>able</b> <small><i>adj.</i></small> suitable<p>', '<a name="abluciouns"><b>abluciouns</b> <small><i>noun</i></small> cleansings<p>', '<a name="abood"><b>abood</b> <small><i>noun</i></small> delay<p>', '<a name="above"><b>above</b> <small><i>adj.</i></small> superior<p>', '<a name="abregge"><b>abregge</b> <small><i>verb</i></small> abridge, shorten<p>']


In [13]:
CTpattern = r"""(?x)
(?P<first_form><a(.*)?\sname=\"(?P<name>.+?)\"><b>(?P<middle_eng_term1>.*?)</b>\s?(<small><i>(?P<pos1>.*?)?</i></small>)?(?P<definition1>.*?))[<;]
(?P<second_form>\s<b>(?P<middle_eng_term2>.*?)</b>\s?(<small><i>(?P<pos2>.*?)?</i></small>)?(?P<definition2>.*?)[<;])?
(?P<extra>.*)?"""

In [14]:
class Fword:
    def __init__(self, name, form, pos, definition, form2=None, pos2=None, def2=None, extra=None):
        self.name = name
        self.form = form
        self.pos = pos
        self.deff = definition
        self.form2 = form2
        self.pos2 = pos2
        self.deff2 = def2
        self.extra = extra
    def __str__(self):
        if True:
            return self.name
        if self.form2:
            return f"fword object of {self.name}, with forms {self.form}, {self.form2}"
        else:
            return f"fword object of {self.name}, with form(s) {self.form}"
    def __repr__(self):
        return f"fwordObject {self.name}"
    def describe(self):
        if self.form2 and self.pos2 and self.def2:
            return f"name: {self.name}, first form: {self.form}, part of speech: {self.pos}, definition: {self.deff}, second form: {self.form2}, part of speech: {self.pos2}, definition: {self.deff2}"
        else:
            return f"name: {self.name}, first form: {self.form}, part of speech: {self.pos}, definition: {self.deff}"
def get_pos(pos, fwords):
    selection = []
    for word in fwords:
        if pos in word.pos or (word.pos2 != None and pos in word.pos2):
            selection.append(word)
    return selection

In [15]:
defined_words = []
x = 0
for line in mylist:
    raw_word = re.search(CTpattern, line)
    try:
        word_name = raw_word['name']
        form1 = raw_word['middle_eng_term1']
        form1_pos = raw_word['pos1']
        if form1_pos == None:
            form1_pos = 'other'
        form1_def = raw_word['definition1']
        try:
            form2 = raw_word['middle_eng_term2']
            form2_pos = raw_word['pos2']
            form2_def = raw_word['definition2']
            try:
                extra = raw_word['extra']
                word = Fword(word_name, form1, form1_pos, form1_def, form2=form2, pos2=form2_pos, def2=form2_def, extra=extra)
            except TypeError as e:
                word = Fword(word_name, form1, form1_pos, form1_def, form2=form2, pos2=form2_pos, def2=form2_def)
        except TypeError as e:
            word = Fword(word_name, form1, form1_pos, form1_def)
        defined_words.append(word)
        x += 1
    except TypeError as e:
        print(x)
        print(e)
        break

2029
'NoneType' object is not subscriptable


In [16]:
print(defined_words[0])

abayst


In [17]:
#defined_words[2]['middle_eng_term1']

In [18]:
defined_words[2]

fwordObject abyde

In [19]:
mylist[1282]

'<a "="" name="peyre of tables"><b>peyre of tables</b> folding set of writing tablets<p>'

In [20]:
mylist[1982]

'<a name="yclawed"><b>yclawed</b><small><i>verb</i></small> lacerated, torn apart<p>'

In [21]:
mylist[1983]

'<a name="ycleped"><b>ycleped (cleped)</b> <small><i>verb, pst. sg.</i></small> called<p>'

In [20]:
mylist[2028]

'<a name="Zephirus"><b>Zephirus</b> <small><i>noun</i></small> the west wind that blows in Spring<p>'

In [21]:
len(get_pos('noun', defined_words))

1070

In [22]:
def get_none(fwords):
    for word in fwords:
        try:
            if 'noun' in word.pos:
                pass
        except TypeError:
            print(word)

In [23]:
get_none(defined_words)

In [24]:
def get_possible_pos(fwords):
    posses = []
    for word in fwords:
        if word.pos not in posses:
            posses.append(word.pos)
        try:
            if word.pos2 not in posses and word.pos2:
                posses.append(word.pos2)
        except AttributeError:
            pass
    return posses

In [25]:
print(get_possible_pos(defined_words))

['other', 'verb', 'verb, prsnt.', 'verb, pst.', 'adj.', 'noun', 'prep.', 'verb, prsnt', 'verb, pst. prtcpl.', 'verb, pst. sg.', 'conj.', 'adv.', 'verb, prsnt. sg.', '(Latin)', 'noun, sg.', 'noun, pl.', 'verb, pst. sg', 'verbal noun', 'verb, 3rd prs. sg.', 'adverb', ' noun', 'verb, pst', 'verb, 3rd prs. prsnt.', 'noun sg.', 'verb, prsnt, 3rd prs. sg.', 'adv. ', 'pst.', 'verb, 1st prs. sg. prst', 'verb, 3rd prs. sg. prst.', 'comp.', '', 'verb, 2nd prs. sg. prsnt.', 'verb, 3rd prs. sg. prsnt.', 'verb, pst. ', 'pro.', 'adj. superlative', 'noun &amp; adj.', 'noun, pl', 'adv. superlative', 'verb, prs. prtcpl.', 'num.', 'noun &amp; verb', 'interj.', 'comparative', 'pro. ', 'adj. ', 'verb. pst.', 'verb, verbal noun', 'noun pl.', '(French)', 'adj. &amp; noun', 'verb, 2nd prs. sg.', 'verb, 3rd prs.sg.', 'verb, prtcpl.', 'adv.</i> </small> closely; <small><i>adj.', 'adv. &amp; noun', 'adj', '3rd prs. sg.', 'adv</i>.</small> 1. gently; 2. quietly; <small><i>adj.', 'conj. &amp; prep.', 'demonstr. a

In [26]:
for i in defined_words:
    for j in ('(', 'comp.', 'pro.', 'num.', 'verbal noun', 'prtcpl.', '<', 'comparative', 'demonstr.', 'p.'):
        if  i.pos == 'pst.' or not i.pos or j in i.pos:
            print(i.describe())
            break

name: adoun, first form: adoun, part of speech: prep., definition:  below
name: after-mete, first form: after-mete, part of speech: prep., definition:  the time after diner
name: agast, first form: agast, part of speech: verb, pst. prtcpl., definition:  frightened
name: amended, first form: amended, part of speech: verb, pst. prtcpl., definition:  improved, corrected
name: amor vincit omnia, first form: Amor vincit omnia, part of speech: (Latin), definition:  Love conquers all
name: arrayed, first form: arrayed, part of speech: verb, pst. prtcpl., definition:  prepared, adorned
name: at erst, first form: at erst, part of speech: prep., definition:  for the first time
name: avauntyng, first form: avauntyng, part of speech: verbal noun, definition:  boasting
name: ave marie, first form: Ave Marie, part of speech: (Latin), definition:  Hail Mary (first words of a Latin prayer)
name: berkyng, first form: berkyng, part of speech: verbal noun, definition:  barking
name: bitwix, first form: b

In [27]:
nouns = get_pos('noun', defined_words)
print(len(nouns))

1070


In [28]:
for noun in nouns[:10]:
    print(noun.describe())

name: abluciouns, first form: abluciouns, part of speech: noun, definition:  cleansings
name: abood, first form: abood, part of speech: noun, definition:  delay
name: accidie, first form: accidie, part of speech: noun, definition:  sloth, laziness
name: accord, first form: accord, acord, part of speech: noun, definition:  agreement
name: achatours, first form: achatours, part of speech: noun, definition:  buyers
name: affiance, first form: affiance, part of speech: noun, definition:  trust
name: agu, first form: agu, part of speech: noun, definition:  acute fever
name: aiel, first form: aiel, part of speech: noun, definition:  grandfather 
name: aketoun, first form: aketoun, part of speech: noun, definition:  wadded jacket worn under the chain-mail coat
name: alauntz, first form: alauntz, part of speech: noun, definition:  wolfhounds


9/26
<h1>What I want:</h1>
a fast method for selecting good words/phrases that's also repeatable  
<h1>How to accomplish that:</h1>

1. A function that displays a word and its info, one at a time, and I hit a button to save it or reject it
2. Literally completely at random
<h2>Option 1</h2>

- hit 1 for save, 2 for reject
- should save my choices when reloading jupyter
- much harder to implement
- more artistic control/input

<h2>Option 2</h2>

- very easy to implement
- little to no control
- may be harder to select phrases for fourth line

First: just write Option 2, just in case Option 1 takes too long

In [29]:
nouns = get_pos('noun', defined_words)
adjectives = get_pos('adj.', defined_words)
participles = get_pos('prtcpl.', defined_words) + get_pos('verbal noun', defined_words)

In [30]:
print(len(adjectives), len(participles))

238 40


In [31]:
print(participles)

[fwordObject agast, fwordObject amended, fwordObject arrayed, fwordObject clad, fwordObject depeint, fwordObject depeynted, fwordObject encombred, fwordObject eschawfed, fwordObject fletynge, fwordObject floytynge, fwordObject kembd, fwordObject kithed, fwordObject mowled, fwordObject mysboden, fwordObject offended, fwordObject rownynge, fwordObject wist, fwordObject ybete, fwordObject yiven, fwordObject avauntyng, fwordObject berkyng, fwordObject chidyng, fwordObject conseillyng, fwordObject eschawfyne, fwordObject galpyng, fwordObject grucchyng, fwordObject herknynge, fwordObject janglerie, fwordObject mordrynge, fwordObject murmur, fwordObject prechyng, fwordObject prikyng, fwordObject rowtyng, fwordObject saluyng, fwordObject stryvyng, fwordObject sublymyng, fwordObject travaillynge, fwordObject unyolden, fwordObject venerie, fwordObject wrastlynge]


In [32]:
from random import choice, choices, randint, sample
import os
import time
import IPython
import IPython.display
from IPython.display import HTML
import json
import base64

In [33]:
class Diamante:
    def __init__(self, words, nouns, adjs, prtcpls):
        self.topic1, self.topic2 = choices(nouns, k=2)
        self.adj1, self.adj2, self.adj3, self.adj4 = choices(adjs, k=4)
        self.p1, self.p2, self.p3, self.p4, self.p5, self.p6 = choices(prtcpls, k=6)
        self.template1 = "{0} & {1}"
        self.template2 = "{0} {1}" #adj + noun (repeated)
        self.phrasenouns = choices(nouns, k=4)
        self.phraseadjs = choices(adjs, k=2)
        if not randint(0, 1):
            self.phrase1 = self.template1.format(*self.phrasenouns[:2])
            self.phrase2 = self.template1.format(*self.phrasenouns[2:])
            self.phrase = f'{self.phrase1}, {self.phrase2}'
            self.phrase_def = f'[{self.phrasenouns[0].deff}] & [{self.phrasenouns[1].deff}], [{self.phrasenouns[2].deff}] & [{self.phrasenouns[3].deff}]'
        else:
            self.phrase1 = self.template2.format(self.phraseadjs[0], self.phrasenouns[0])
            self.phrase2 = self.template2.format(self.phraseadjs[1], self.phrasenouns[1])
            self.phrase = f'{self.phrase1} and {self.phrase2}'
            self.phrase_def = f'[{self.phraseadjs[0].deff}] [{self.phrasenouns[0].deff}] and [{self.phraseadjs[1].deff}] [{self.phrasenouns[1].deff}]'
    def makePoem(self):
        print()
        display(HTML(f'<center><font face="Canterbury" size="4"> {self.topic1.name} </font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.adj1.name}  and {self.adj2.name}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.p1.name}, {self.p2.name}, {self.p3.name}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4">{self.phrase}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.p4.name}, {self.p5.name}, {self.p6.name}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.adj3.name}  and {self.adj4.name}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.topic2.name} </font></center>'))
        print()
        display(HTML(f'<center><font face="garamond" size="4"> [{self.topic1.deff}] </font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.adj1.deff}]  and [{self.adj2.deff}]</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.p1.deff}], [{self.p2.deff}], [{self.p3.deff}]</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4">{self.phrase_def}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.p4.deff}], [{self.p5.deff}], [{self.p6.deff}]</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.adj3.deff}]  and [{self.adj4.deff}]</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.topic2.deff}] </font></center>'))
        print()
        if input("Do you want to save this poem?").lower() in ('y', 'yes'):
            save(self)

In [22]:
class PoemEncoder(json.JSONEncoder):
    """ An encoder for Diamante poems (objects).
    """
    def default(self, obj):
        """ Encode a Diamante poem.
        
        Arguments:
            obj (Diamante): the poem to encode
            
        Returns:
            dict: the Diamante object's attributes, as a dictionary
            if obj is not a Diamante, return the default method of the parent class
        """
        if isinstance(obj, Diamante) or isinstance(obj, Diamante2):
            return {'topics': (obj.topic1, obj.topic2),
                    'adjectives': (obj.adj1, obj.adj2, obj.adj3, obj.adj4),
                    'participles': (obj.p1, obj.p2, obj.p3, obj.p4, obj.p5, obj.p6),
                    'phrase': (obj.phrase),
                    'phrase definition': (obj.phrase_def)
                    }
        return json.JSONEncoder.default(self, obj)

class FwordEncoder(json.JSONEncoder):
    """ An encoder for Fword (objects).
    """
    def default(self, obj):
        """ Encode an Fword object.
        
        Arguments:
            obj (Fword): the word to encode
            
        Returns:
            dict: the Fword object's attributes, as a dictionary
            if obj is not an Fword, return the default method of the parent class
        """
        if isinstance(obj, Fword):
            return {'name': obj.name,
                    'form': obj.form,
                    'part of speech': obj.pos,
                    'definition': obj.deff
                    }
        return json.JSONEncoder.default(self, obj)

def save(poem, path='goodPoem'):
    with open(f'{path}.json', 'w', encoding='utf-8') as f:
        newattrs = [json.dumps(word, indent=2, cls=FwordEncoder) for word in (poem.topic1, poem.topic2, poem.adj1, poem.adj2, poem.adj3, poem.adj4, poem.p1, poem.p2, poem.p3, poem.p4, poem.p5, poem.p6)]
        #newattrs.append(poem.phrase, poem.phrase_def)
        poem.topic1, poem.topic2, poem.adj1, poem.adj2, poem.adj3, poem.adj4, poem.p1, poem.p2, poem.p3, poem.p4, poem.p5, poem.p6 = newattrs
        json.dump(poem, f, indent=2, cls=PoemEncoder)

def loadFword(word):
    #print(word)
    #print(type(word))
    return Fword(word['name'], word['form'], word['part of speech'], word['definition'])

def load(path, *args, **kwargs):
    with open(path, 'r', encoding='utf-8') as f:
        poemInfo = json.load(f)
    newPoem = Diamante(*args)
    newTopics, newAdjs, newPs = [], [], []
    newPoem.topic1, newPoem.topic2 = [loadFword(json.loads(topic)) for topic in poemInfo['topics']]
    newPoem.adj1, newPoem.adj2, newPoem.adj3, newPoem.adj4 = [loadFword(json.loads(adj)) for adj in poemInfo['adjectives']]
    newPoem.p1, newPoem.p2, newPoem.p3, newPoem.p4, newPoem.p5, newPoem.p6 = [loadFword(json.loads(p)) for p in poemInfo['participles']]
    newPoem.phrase = poemInfo['phrase']
    newPoem.phrase_def = poemInfo['phrase definition']
    return newPoem

In [35]:
choicePoem = Diamante(defined_words, nouns, adjectives, participles)

In [23]:
#choicePoem.makePoem()

In [37]:
class Diamante2:
    def __init__(self, words, nouns, adjs, prtcpls):
        self.topic1, self.topic2 = sample(nouns, k=2)
        self.adj1, self.adj2, self.adj3, self.adj4 = sample(adjs, k=4)
        self.p1, self.p2, self.p3, self.p4, self.p5, self.p6 = sample(prtcpls, k=6)
        self.template1 = "{0} & {1}"
        self.template2 = "{0} {1}" #adj + noun (repeated)
        self.phrasenouns = sample(nouns, k=4)
        self.phraseadjs = sample(adjs, k=2)
        if not randint(0, 1):
            #print(self.phrasenouns[:2])
            self.phrase1 = self.template1.format(*self.phrasenouns[:2])
            self.phrase2 = self.template1.format(*self.phrasenouns[2:])
            self.phrase = f'{self.phrase1}, {self.phrase2}'
            self.phrase_def = f'[{self.phrasenouns[0].deff}] & [{self.phrasenouns[1].deff}], [{self.phrasenouns[2].deff}] & [{self.phrasenouns[3].deff}]'
        else:
            self.phrase1 = self.template2.format(self.phraseadjs[0], self.phrasenouns[0])
            self.phrase2 = self.template2.format(self.phraseadjs[1], self.phrasenouns[1])
            self.phrase = f'{self.phrase1} and {self.phrase2}'
            self.phrase_def = f'[{self.phraseadjs[0].deff}] [{self.phrasenouns[0].deff}] and [{self.phraseadjs[1].deff}] [{self.phrasenouns[1].deff}]'
    def makePoem(self):
        print()
        #display(HTML('<style>{}</style>'.format(mycss)))
        display(HTML(f"<center>{self.topic1.name}</center>"))
        display(HTML(f'<center>{self.adj1.name}  and {self.adj2.name}</center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.p1.name}, {self.p2.name}, {self.p3.name}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4">{self.phrase}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.p4.name}, {self.p5.name}, {self.p6.name}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.adj3.name}  and {self.adj4.name}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> {self.topic2.name} </font></center>'))
        print()
        display(HTML(f'<center><font face="garamond" size="4"> [{self.topic1.deff}] </font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.adj1.deff}]  and [{self.adj2.deff}]</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.p1.deff}], [{self.p2.deff}], [{self.p3.deff}]</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4">{self.phrase_def}</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.p4.deff}], [{self.p5.deff}], [{self.p6.deff}]</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.adj3.deff}]  and [{self.adj4.deff}]</font></center>'))
        display(HTML(f'<center><font face="garamond" size="4"> [{self.topic2.deff}] </font></center>'))
        print()
        if input("Do you want to save this poem?").lower() in ('y', 'yes'):
            save(self)

In [38]:
samplingPoem = Diamante2(defined_words, nouns, adjectives, participles)

In [3]:
#mycss = """
#@font-face: {font-family: Canterbury; src: url('Canterbury.ttf');
#}
#center {
#font-family: Canterbury, Wingdings
#}
#"""

^^^The above cell^^^

Had to be commented out because it affects all cells in all files, and since I wrote an improved version lower down (with some help from ChatGPT), it overrides that cell. However, I want to keep the old cell around for documentation's sake

In [40]:
#samplingPoem.makePoem()

In [41]:
loadTest = load('goodPoem_1.json', defined_words, nouns, adjectives, participles)

In [42]:
#loadTest.makePoem()

In [43]:
loadTest2 = load('goodPoem_2.json', defined_words, nouns, adjectives, participles)

In [44]:
#loadTest2.makePoem()

<style>
    @font-face: {font-family: Canterbury; src: url('Canterbury.ttf');
                }
    center {
        font-family: Canterbury
            }
</style>
<center>hello</center>
<center><font face='Canterbury' size='4'>hello</font></center>

In [45]:
display(HTML('<center>hello</center>'))
display(HTML('<head><style>{}</style></head><center>HELLO</center>'.format(mycss)))

NameError: name 'mycss' is not defined

<center>hello</center>
<center><font face='Canterbury' size='4'>hello</font></center>

NOTE: Originally the previous text was not in the custom font, and I spent hours trying to get it to work. Eventually I asked ChatGPT for help, and put its results into a separate ipynb file to test. Not only did the code work, but it immediately changed the text in this file to the correct font. Also, I don't actually have to add the style to the display(HTML()) call, I just create a variable that stores my desired CSS in a string, and Notebook automatically applies it to all cells in all tabs. Very strange

WORKING VERSION

just the poetry generation itself

In [46]:
class DiamanteFinal:
    def __init__(self, words, nouns, adjs, prtcpls):
        self.topic1, self.topic2 = sample(nouns, k=2)
        self.adj1, self.adj2, self.adj3, self.adj4 = sample(adjs, k=4)
        self.p1, self.p2, self.p3, self.p4, self.p5, self.p6 = sample(prtcpls, k=6)
        self.template1 = "{0} & {1}"
        self.template2 = "{0} {1}" #adj + noun (repeated)
        self.phrasenouns = sample(nouns, k=4)
        self.phraseadjs = sample(adjs, k=2)
        if not randint(0, 1):
            #print(self.phrasenouns[:2])
            self.phrase1 = self.template1.format(*self.phrasenouns[:2])
            self.phrase2 = self.template1.format(*self.phrasenouns[2:])
            self.phrase = f'{self.phrase1}, {self.phrase2}'
            self.phrase_def = f'[{self.phrasenouns[0].deff}] & [{self.phrasenouns[1].deff}], [{self.phrasenouns[2].deff}] & [{self.phrasenouns[3].deff}]'
        else:
            self.phrase1 = self.template2.format(self.phraseadjs[0], self.phrasenouns[0])
            self.phrase2 = self.template2.format(self.phraseadjs[1], self.phrasenouns[1])
            self.phrase = f'{self.phrase1} and {self.phrase2}'
            self.phrase_def = f'[{self.phraseadjs[0].deff}] [{self.phrasenouns[0].deff}] and [{self.phraseadjs[1].deff}] [{self.phrasenouns[1].deff}]'
    def makePoem(self):
        print()
        #display(HTML('<style>{}</style>'.format(mycss)))
        display(HTML(f"<div class='original_text'><center>{self.topic1.name}</center></div>"))
        display(HTML(f"<div class='original_text'><center>{self.adj1.name}  and {self.adj2.name}</center></div>"))
        display(HTML(f"<div class='original_text'><center>{self.p1.name}, {self.p2.name}, {self.p3.name}</center></div>"))
        display(HTML(f"<div class='original_text'><center>{self.phrase}</center></div>"))
        display(HTML(f"<div class='original_text'><center>{self.p4.name}, {self.p5.name}, {self.p6.name}</center></div>"))
        display(HTML(f"<div class='original_text'><center>{self.adj3.name}  and {self.adj4.name}</center></div>"))
        display(HTML(f"<div class='original_text'><center>{self.topic2.name}</center></div>"))
        print()
        display(HTML(f"<div class='translation'><center>[{self.topic1.deff}]</center></div>"))
        display(HTML(f"<div class='translation'><center>[{self.adj1.deff}]  and [{self.adj2.deff}]</center></div>"))
        display(HTML(f"<div class='translation'><center>[{self.p1.deff}], [{self.p2.deff}], [{self.p3.deff}]</center></div>"))
        display(HTML(f"<div class='translation'><center>{self.phrase_def}</center></div>"))
        display(HTML(f"<div class='translation'><center>[{self.p4.deff}], [{self.p5.deff}], [{self.p6.deff}]</center></div>"))
        display(HTML(f"<div class='translation'><center>[{self.adj3.deff}]  and [{self.adj4.deff}]</center></div>"))
        display(HTML(f"<div class='translation'><center>[{self.topic2.deff}]</center></div>"))
        print()
        if input("Do you want to save this poem?").lower() in ('y', 'yes'):
            save(self)

In [1]:
class PoemEncoder(json.JSONEncoder):
    """ An encoder for Diamante poems (objects).
    """
    def default(self, obj):
        """ Encode a Diamante poem.
        
        Arguments:
            obj (Diamante): the poem to encode
            
        Returns:
            dict: the Diamante object's attributes, as a dictionary
            if obj is not a Diamante, return the default method of the parent class
        """
        if isinstance(obj, DiamanteFinal):
            return {'topics': (obj.topic1, obj.topic2),
                    'adjectives': (obj.adj1, obj.adj2, obj.adj3, obj.adj4),
                    'participles': (obj.p1, obj.p2, obj.p3, obj.p4, obj.p5, obj.p6),
                    'phrase': (obj.phrase),
                    'phrase definition': (obj.phrase_def)
                    }
        return json.JSONEncoder.default(self, obj)

class FwordEncoder(json.JSONEncoder):
    """ An encoder for Fword (objects).
    """
    def default(self, obj):
        """ Encode an Fword object.
        
        Arguments:
            obj (Fword): the word to encode
            
        Returns:
            dict: the Fword object's attributes, as a dictionary
            if obj is not an Fword, return the default method of the parent class
        """
        if isinstance(obj, Fword):
            return {'name': obj.name,
                    'form': obj.form,
                    'part of speech': obj.pos,
                    'definition': obj.deff
                    }
        return json.JSONEncoder.default(self, obj)

def save(poem, path='goodPoem'):
    with open(f'{path}.json', 'w', encoding='utf-8') as f:
        newattrs = [json.dumps(word, indent=2, cls=FwordEncoder) for word in (poem.topic1, poem.topic2, poem.adj1, poem.adj2, poem.adj3, poem.adj4, poem.p1, poem.p2, poem.p3, poem.p4, poem.p5, poem.p6)]
        #newattrs.append(poem.phrase, poem.phrase_def)
        poem.topic1, poem.topic2, poem.adj1, poem.adj2, poem.adj3, poem.adj4, poem.p1, poem.p2, poem.p3, poem.p4, poem.p5, poem.p6 = newattrs
        json.dump(poem, f, indent=2, cls=PoemEncoder)

def loadFword(word):
    #print(word)
    #print(type(word))
    return Fword(word['name'], word['form'], word['part of speech'], word['definition'])

def load(path, *args, **kwargs):
    with open(path, 'r', encoding='utf-8') as f:
        poemInfo = json.load(f)
    newPoem = DiamanteFinal(*args)
    newTopics, newAdjs, newPs = [], [], []
    newPoem.topic1, newPoem.topic2 = [loadFword(json.loads(topic)) for topic in poemInfo['topics']]
    newPoem.adj1, newPoem.adj2, newPoem.adj3, newPoem.adj4 = [loadFword(json.loads(adj)) for adj in poemInfo['adjectives']]
    newPoem.p1, newPoem.p2, newPoem.p3, newPoem.p4, newPoem.p5, newPoem.p6 = [loadFword(json.loads(p)) for p in poemInfo['participles']]
    newPoem.phrase = poemInfo['phrase']
    newPoem.phrase_def = poemInfo['phrase definition']
    return newPoem

NameError: name 'json' is not defined

In [48]:
font_path = 'Canterbury.ttf'
with open(font_path, 'rb') as font_file:
    font_data = font_file.read()
    encoded_font = base64.b64encode(font_data).decode('utf-8')

In [49]:
mycss2 = f"""
@font-face {{
    font-family: 'Canterbury';
    src: url(data:font/ttf;base64,{encoded_font}) format('truetype');
}}

.original_text {{
    font-family: 'Canterbury', serif;
    font-size: 24px;
}}

.translation {{
    font-family: 'Garamond', serif;
    font-size: 20px;
}}
"""


In [50]:
display(HTML(f"<style>{mycss2}</style>"))

In [61]:
workingPoem = DiamanteFinal(defined_words, nouns, adjectives, participles)

In [62]:
workingPoem.makePoem()

Do you want to save this poem? yes


In [63]:
loadTest = load('goodPoem.json', defined_words, nouns, adjectives, participles)

In [65]:
loadTest.makePoem()

Do you want to save this poem? no
